In [ ]:
from gpkit import Model, Variable, units
from gpkit.shortcuts import Var, Model
from cost import DAPCA4Cost
from breguet_range import BreguetRange

class RvsC(Model):
    def setup(self):
        cost = DAPCA4Cost()
        rng = BreguetRange()
        
        C_fly = cost["C_{fly}"]
        R = rng["R"]
        V = rng["V"]
        
        R_min = Var("R_{min}", 2000, "nautical_miles", "Minimum Range")
        V_min = Var("V_{min}", 420, "knots", "Minimum Velocity")
        objective = C_fly # minimize cost
        constraints = [R >= R_min,
                       V >= V_min]
        
        model = Model(objective, constraints)

        for subm in [cost, rng]:
            model = model & subm
            
        return model
    
    def test(self):
        self.solve(verbosity=0)
    
#print RvsC().solve(verbosity=0).table(["cost", "freevariables"])
prob = RvsC()
sol = prob.solve()


In [ ]:
import gpkit.interactive
gpkit.interactive.showcadtoon("interactive/BlankMapLowRes",
                              "position:absolute; height:0em; right:42em; top:42em;")

In [ ]:
oldssubs = dict(prob.substitutions)

In [ ]:
from interactive import ractorpy_output
# RANGES={'W_{payload}': (100, 900, 50),
#         'T_{loiter}': (0.02, 10, (10-.02)/10.),
#         'T_{reserve}': (5, 90, 10),
#         'MTOW': (2500, 5000, 200),
#         'M_{min-outbound}': (0.66, 0.87, .02),
#         'cs': (0.1, 0.5, 0.02),
#         'R_{min}': (500, 1000, 100)}
# gpkit.interactive.ractorpy(prob, ractorpy_output, RANGES,
#                            showtables=None)

def fn_of_sol(sol):
    print sol.table(["cost"])
    ractorpy_output(sol)

RANGES={'{R_{min}}_{RvsC}': (100,200,10)}
gpkit.interactive.ractorpy(prob, ractorpy_output, None,
                           showtables=None)

In [ ]:
prob.substitutions

In [ ]:
prob.solve()

In [ ]:
prob.substitutions = dict(oldssubs)
prob.solve()

In [ ]:
% pylab inline
RANGES={'R_{min}': (100,600,10)}
from gpkit.interactive.plotting import sensitivity_plot
_ = sensitivity_plot(prob, keys=RANGES)